In [ ]:
!apt-get update && apt-get install -y libprotobuf-dev protobuf-compiler

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 2s (1,797 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [ ]:
!pip install pycld3==0.21

In [ ]:
import os, zipfile, requests, json

# Download benchmark test set
url = "https://github.com/AI4Bharat/IndicLID/releases/download/v1.0/bhasha-abhijnaanam_test_set.zip"
os.makedirs("data", exist_ok=True)
zip_path = "data/bhasha_test.zip"

if not os.path.exists("data/bhasha_test"):
    print("Downloading benchmark test set...")
    r = requests.get(url)
    with open(zip_path, "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("data/")
    os.remove(zip_path)
    print("Zip file removed.")

# Inspect files
os.listdir("data/")

import json
import pandas as pd

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)   # one JSON object
    data = obj["data"]

    # build DataFrame
    df = pd.DataFrame([{
        "id": d["unique_identifier"],
        "native": d["native sentence"],
        "roman": d["romanized sentence"],
        "label": d["language"],
        "script": d["script"],
        "source": d["source"]
    } for d in data])

    return df

# usage
df = load_json("data/bhasha-abhijnaanam.json")

print(df.shape)
df.sample(2)

native_test = df[['native', 'label']]

native_test = native_test[native_test['native'] != '']

native_test['label'].unique()

native_test[native_test['label'] == 'Hindi'].sample(1).iloc[0].native

native_test.sample()

!pip install indic-nlp-library --quiet

from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

lang_code_map = {
    'Assamese': 'as',
    'Bodo': 'brx',
    'Bangla': 'bn',
    'Konkani': 'gom',
    'Gujarati': 'gu',
    'Hindi': 'hi',
    'Kannada': 'kn',
    'Maithili': 'mai',
    'Malayalam': 'ml',
    'Marathi': 'mr',
    'Nepali': 'ne',
    'Oriya': 'or',
    'Punjabi': 'pa',
    'Sanskrit': 'sa',
    'Sindhi': 'sd',
    'Tamil': 'ta',
    'Telugu': 'te',
    'Urdu': 'ur',
    'Kashmiri': 'ks',
    'Manipuri': 'mni',
    'Dogri': 'doi',
    'Santali': 'sat'  # ⚠️ Santali is Ol Chiki script, may not be fully supported in IndicNLP
}

import unicodedata

def fallback_normalize(text):
    return " ".join(unicodedata.normalize("NFKC", text).split())


def preprocess_native_text(df):
    processed_texts = []
    factory = IndicNormalizerFactory()

    for idx, row in df.iterrows():
        text = row['native']
        label = row['label']
        lang = lang_code_map.get(label, 'hi')  # fallback Hindi if unknown

        try:
            # Create normalizer for the specific language
            normalizer = factory.get_normalizer(lang)
            norm_text = normalizer.normalize(text)

            # Tokenize
            tokens = indic_tokenize.trivial_tokenize(norm_text, lang)
            processed_texts.append(" ".join(tokens))

        except Exception as e:
            # If not supported in IndicNLP, just keep raw text
            print(f"⚠️ Skipping normalization for {label} (lang={lang}): {e}")
            norm_text = fallback_normalize(text)
            processed_texts.append(text)

    return processed_texts

native_test['processed'] = preprocess_native_text(native_test)
native_test['true_lang_code'] = native_test['label'].map(lang_code_map)

# native_test.to_csv('native_test.csv', index=False)

# Sample one row where label is 'Hindi'
sample_row = native_test[native_test['label'] == 'Hindi'].sample(1).iloc[0]
native_text, processed_text = sample_row.iloc[0], sample_row.iloc[2]



Streaming output truncated to the last 5000 lines.
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipuri (lang=mni): 'mni'
⚠️ Skipping normalization for Manipur

In [ ]:
import cld3
import pandas as pd
import time

# Load your CSV
native_test = pd.read_csv("/content/native_test.csv")

true_labels = native_test.iloc[:, 1].astype(str)

texts = native_test.iloc[:, 2].astype(str)
texts=texts.tolist()

indic_langs = {
    "as": "Assamese", "bn": "Bengali", "gu": "Gujarati", "hi": "Hindi", "kn": "Kannada",
    "ml": "Malayalam", "mr": "Marathi", "ne": "Nepali", "or": "Odia", "pa": "Punjabi",
    "sd": "Sindhi", "si": "Sinhala", "ta": "Tamil", "te": "Telugu", "ur": "Urdu",
    "ks": "Kashmiri", "mai": "Maithili", "gom": "Konkani", "mni": "Manipuri",
    "sa": "Sanskrit", "brx": "Bodo"
}

def detect_language(text):
    result = cld3.get_language(text)
    lang = result.language if result is not None else "other"
    return lang if lang in indic_langs else "other"

# Measure time and predict
start_time = time.time()
predicted_lang = [detect_language(text) for text in texts]
end_time = time.time()

print(f"Throughput: {len(texts)/(end_time-start_time)} sentence/seconds")

Throughput: 3248.88965959067 sentence/seconds


In [ ]:
true_labels = true_labels.apply(lambda x: x if x in indic_langs.values() else "other")

In [ ]:
# import time
# start_time=time.time()
# predicted_lang = [detect_language(text) for text in native_test['processed'].to_list()]
# end_time=time.time()
y_true = true_labels.map(lang_code_map)
y_pred = predicted_lang
y_true=y_true.tolist()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


cld3_accuracy = accuracy_score(y_true, y_pred)
cld3_precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
cld3_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
cld3_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print("cld3_Native Accuracy:", cld3_accuracy*100)
print("cld3_Native Precision (macro):", cld3_precision*100)
print("cld3_Native Recall (macro):", cld3_recall*100)
print("cld3_Native F1-score (macro):", cld3_f1*100)
print(f"cld3_Throughput: {len(texts)/(end_time-start_time)} sentence/seconds")

cld3_Native Accuracy: 68.10393800986206
cld3_Native Precision (macro): 43.220697053236066
cld3_Native Recall (macro): 51.320840445985596
cld3_Native F1-score (macro): 45.76828351559491
cld3_Throughput: 3248.88965959067 sentence/seconds
